In [3]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
    sys.path.append("../")
from lib.envs.gridworld import GridworldEnv
    

In [4]:
pp = pprint.PrettyPrinter(indent = 2)
env = GridworldEnv()

In [5]:
def policy_eval(policy, env, discount_factor = 1.0, theta = 0.00001):
    #start with random value function
    V = np.zeros(env.nS)
    
    while True:
        delta = 0
        for s in range(env.nS):
            v = 0
            #look at the possible next action
            for a, action_prob in enumerate(policy[s]):
                #for each action look at the possible next state...
                for prob, next_state, reward, done in env.P[s][a]:
                    #calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
                    #how much our value function change
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        if delta < theta:
            break
    return np.array(V)
                

In [6]:
def policy_improvement(env, policy_eval_fn = policy_eval, discount_factor = 1.0):
    def one_step_lookahead(state, V):
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    #start with random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        #evaluate the current policy
        V = policy_eval_fn(policy, env, discount_factor)
        
        policy_stable = True
        
        #for each state
        for s in range(env.nS):
            #the best action we would take under current policy
            chosen_a = np.argmax(policy[s])
            action_values = one_step_lookahead(s, V)
            best_a = np.argmax(action_values)
            
            #greedly update the policy
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(env.nA)[best_a]
            
        if policy_stable:
            return policy, V

In [7]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution: ")
print(policy)
print("")

print("Reshaped Grid policy (0=up, 1=right, 2=down, 4=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Reshaped grid value function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution: 
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid policy (0=up, 1=right, 2=down, 4=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Reshaped grid value function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [8]:
#test of the value function
expected_v = np.array([0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)